# Miniproject: Deep Reinforcement Learning for Flappy Bird
* MDP Formulation: $(S,A,P_{a},R_{a})$
* Q-Learning and Deep Q-Learning
* Used DQN architecture
* Training and results
* Conclusion and Outlook



# Markov Decision Process Formulation
### FlappyBird Environment
* Goal: control the bird and stay alive (and beat the high score)
* published in 2013
* 50.000$ a day with in-app advertisement

<video controls src="assets/test.mkv" />



FlappyBird game using pygame platform

FPS: 30 

observation: current_screen, reward, terminal_state

### Action definition:

ActionSpace = A = {1 (flap), 0 (drop)}

### State definition:

$s_t = x_{t-histLen+1} ,\; a_{t-histLen+1},\; ... ,\; x_{t-1},\; a_{t-1},\; x_t$

with $x_t: $ pixel input

$a_t: $ action taken

$histLen$: number of stack of observations (temporal information)

### Transitions:
unknown

### Reward:
unknown

### Ingame Reward Definitions

| Name  | reward |
|---:|:-------------|
| rewardAlive | 0.1  | 
| rewardPipe |  1  | 
| rewardDead | -1  | 

$R_t = r_t + \gamma * R_{t+1}$

with $\gamma$ as discount factor




# Q-Learning / Deep Q-Learning (DQN)

### Q-Learning:
* model-free (estimate the Q-function)
* Q(s, a) represents the estimate of future rewards
* find $\pi^*$ by choosing action with highest q-value
* off policy (finds $\pi^*$ independent of behavioral policy)

<img src="assets/q-learning_table.png" width= "800"> Figure from [1] 

${Q^{\pi^{*}}(s, a) = r + \gamma * max_{a^{'}} Q(s^{'}, a^{'})}$

Update Q-Values with temporal difference step:

$Q(s, a)_{new} \leftarrow Q(s, a)_{old} + \alpha (y - Q(s, a)_{old})$

with $y = r + \gamma max_a^{'} Q(s^{'}, a^{'})_{old}$



#### What if the state space is too large?
- Go: $10^{170}$
- continues state space in helicopter flying

### Function approximator for Q-function
$Q(s, a, \textbf{w}) \approx Q(s, a)$

<img src="assets/dqn_network.png" width="800"> Figure from [1] 

$\mathcal{L_i(\mathbf{w_i})} = \mathbb{E}_{s,a \sim ρ(s,a)}\big[\big(y -  Q(s, a, \mathbf{w_i})\big)^2 \big] $

$y_i = \mathbb{E}_{s^{'} \sim \varepsilon} \big[r + \gamma * max_{a^{'}} Q(s^{'}, a^{'}, \mathbf{w_{i}})\big]$
### Problem: 
* non-stationary target: changing the weights changes the target
* trajectories are correlated (we want i.i.d.)

$\rightarrow$ congergence issues in training the network


####  Mnih et al., 2013, Playing Atari with Deep Reinforcement Learning
#### Mnih et al., 2015, Human-level control through deep reinforcement learning
## Expierence replay
tuple $expierence =  (s, a, r, s')$

store in replay memory $\mathcal{D}$

* REPLAY_MEMORY = 50000 
* sample a minibatch of all expieriences in the replay memory

$\rightarrow$ decorrelates the observations


## Fixed target network
* target and predictor network are the same but with different weights
* freeze the target for c-th iterations
* assign current weights to target network


$\mathcal{L_i(\mathbf{w_i})} = \mathbb{E}_{s,a,r,s^{'} \sim \mathcal{D}}\big[\big(r + \gamma * max_{a^{'}} Q(s^{'}, a^{'}, \mathbf{w_{i-c}}) -  Q(s, a, \mathbf{w_i})\big)^2 \big] $





# Used DQN architecture


## Image Preprocessing
* set background to black
<img src="assets/imgProcess_diagramm.png">
<img src="assets/preprocessing.png" width="600">



# CNN

Trainable weights: 3,356,322
<img src="assets/model.png" width="300">

## Game difficulties

| difficulty  | pipeGapSize |
|---:|:-------------|
| easy | 200   | 
| medium | 150   | 
| hard | 100 | 

<img src="assets/pipeSize.png" width="500">



# Explorations vs Exploitation: Epsilon-greedy 
* decay epsilon over time

* generaly start with INITIAL_EPSILON = 1.0
* agent can choose an action every 33ms (FPS=30): 
 * action "flap" changes accleration by value of -9
 * action "do nothing" changes accleration by value of 1


<img src="assets/epsilon.png">


## Training

* arround 15-17h on a GPU
* start training after 1000 obervations
* loss: Mean squared error
* optimizer= Adam
* batchsize = 32
* decay rate $\gamma$ = 0.95 
* learning rate = $10^{-4}$


## Learning plots for "easy"

<img src="assets/learning_plots.png">

# Results

## Mean score of DQN for different difficulties and different learning iterations

* score results are based on the mean of 10 games with epsilon = 0
* terminate game after score passed 1000 points which is regarded as infinity (inf)


| # iterations  | easy | medium | hard |
|---:|:-------------|-------|-------|
| 990000 | inf  |  231.2  |  18.3  | 
| 199000 | 930.3   | 913.9  |  22.5  | 
| 299000 | 223.8   |  105.4  |  197.7  | 
| 399000 |  795.1  |  21.0  |  91.7  | 
| 499000 |  275.5  |  14.4 |  30.7  |


# How good are the trained models for the other difficulties?

### trained on "easy"

| # iterations  | easy | medium | hard |
|---:|:-------------|-------|-------|
| 990000 | inf  |  10.9  |  0.3  | 
| 199000 | 930.3   | 1.7  |  0.0  | 
| 299000 | 223.8   |  7.3  |  0.2  | 
| 399000 |  795.1  |  1.1  |  0.2  | 
| 499000 |  275.5  |  4.1 |  0.1  |

### trained on "medium"

| # iterations  | easy | medium | hard |
|---:|:-------------|-------|-------|
| 990000 | inf  |  231.2  |  2.3  | 
| 199000 | 133.2   | 913.9  |  1.2  | 
| 299000 | inf  |  105.4  |  0.5  | 
| 399000 |  682.4  |  21.0  |  0.9  | 
| 499000 |  62.4  |  14.4 |  0.3  |



### trained on "hard"

| # iterations  | easy | medium | hard |
|---:|:-------------|-------|-------|
| 990000 | 6.8  |  4.2  |  18.3  |
| 199000 | 233.3   | 64.7  |  22.5  | 
| 299000 | 57.4  |  287.4  |  197.7  | 
| 399000 |  156.8  |  165.1  |  91.7  | 
| 499000 |  23.5  |  50.8 |  30.7  |


## Comparing Network with and without Fixed Target Network
##### not fixed target network
<img src="assets/scores_10k.png">

##### fixed target network
<img src="assets/fixedTarget_hard_scores_10k.png">

## Comparing Training with and without rewardAlive

| # iterations  | easy | easy with rewardAlive | medium | medium with rewardAlive | hard | hard with rewardAlive |
|---:|:-------------|-------|-------|-----|-------|-----|
| 990000 | 999.5   |  <strong>inf</strong>  |  95.0  | <strong>231.2</strong>  | 3.1  | <strong>18.3</strong>
| 199000 | 55.0   |  <strong>930.3</strong>  |  7.1  | <strong>913.9</strong>  | 9.6  | <strong>22.5</strong>
| 299000 | 47.3  |  <strong>223.8</strong>  |  20.6 | <strong>105.4</strong>  | 26.3 | <strong>197.7</strong>
| 399000 | 43.5  |  <strong>795.1</strong>  |  <strong>85.0</strong>  | 21.0  | <strong>116.5</strong> | 91.7
| 499000 |  87.5  |  <strong>275.5</strong>  |  <strong>74.7</strong>  | 14.4  | 24.9 | <strong>30.7</strong>

## Conclusion
* DQN is working for different difficulties and is reaching high scores (better as human player)
* 500.000 learning steps are not necessary
* training is "stable" witout fixed target network
* rewardAlive gives a slight advantage (no sparse rewards)


* Same architecture can be used for different games


## Problems and Outlook
* training is not consistent (more training does not correlate with better score)
 * catastrophic forgetting
 * save the network parameters that resulted in the best test performance
 * clipping the gradient between −1.0 and 1.0 (no drastic weight changes by single mini-batch)


* minibatches are sampled uniformly from replay buffer 
 * Prioritized Experience Replay (pay more attention to samples with large target gap)



* explore fixed target network
* use of Dueling DQN Architecture



## References
[1] https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc